In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import cv2

In [2]:
df = pd.read_csv('plates.csv')
df.head()

,filepath,xmin,xmax,ymin,ymax
0,data\annotations\Cars0.xml,226,419,125,173
1,data\annotations\Cars1.xml,134,262,128,160
2,data\annotations\Cars10.xml,140,303,5,148
3,data\annotations\Cars100.xml,175,214,114,131
4,data\annotations\Cars101.xml,167,240,202,220


In [4]:
import xml.etree.ElementTree as xet

In [8]:
filename = df['filepath'][0]

In [12]:
def getfilename(filename):
    filename_image = xet.parse(filename).getroot().find('filename').text
    filepath_image = os.path.join("data\\images",filename_image)
    return filepath_image


In [13]:
getfilename(filename)

'data\\images\\Cars0.png'

In [16]:
image_path = list(df['filepath'].apply(getfilename))
image_path

['data\\images\\Cars0.png',
 'data\\images\\Cars1.png',
 'data\\images\\Cars10.png',
 'data\\images\\Cars100.png',
 'data\\images\\Cars101.png',
 'data\\images\\Cars102.png',
 'data\\images\\Cars103.png',
 'data\\images\\Cars104.png',
 'data\\images\\Cars105.png',
 'data\\images\\Cars106.png',
 'data\\images\\Cars107.png',
 'data\\images\\Cars108.png',
 'data\\images\\Cars109.png',
 'data\\images\\Cars11.png',
 'data\\images\\Cars110.png',
 'data\\images\\Cars111.png',
 'data\\images\\Cars112.png',
 'data\\images\\Cars113.png',
 'data\\images\\Cars114.png',
 'data\\images\\Cars115.png',
 'data\\images\\Cars116.png',
 'data\\images\\Cars117.png',
 'data\\images\\Cars118.png',
 'data\\images\\Cars119.png',
 'data\\images\\Cars12.png',
 'data\\images\\Cars120.png',
 'data\\images\\Cars121.png',
 'data\\images\\Cars122.png',
 'data\\images\\Cars123.png',
 'data\\images\\Cars124.png',
 'data\\images\\Cars125.png',
 'data\\images\\Cars126.png',
 'data\\images\\Cars127.png',
 'data\\images\\C

In [20]:
file_path = image_path[0]
file_path

'data\\images\\Cars0.png'

In [24]:
img = cv2.imread(file_path)

cv2.namedWindow('example',cv2.WINDOW_NORMAL)
cv2.imshow('example',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [26]:
#226,419,125,173

cv2.rectangle(img,(226,125),(419,173),(0,0,255),3)
cv2.namedWindow('example',cv2.WINDOW_NORMAL)
cv2.imshow('example',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

Preprocessing

In [34]:
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import load_img, img_to_array

In [49]:
labels = df.iloc[:,1:]

In [56]:
data = []
output = []
for ind in range(len(image_path)):
    image = image_path[0]
    img_arr = cv2.imread(image)
    h,w,d = img_arr.shape
    
    load_image = load_img(image,target_size=(224,224))
    load_image_arr = img_to_array(load_image)
    norm_load_image_arr = load_image_arr/255.0
    
    xmin = labels['xmin'][ind]
    xmax = labels['xmax'][ind]
    ymin = labels['ymin'][ind]
    ymax = labels['ymax'][ind]
    
    nxmin,nxmax = xmin/w, xmax/w
    nymin, nymax = ymin/h, ymax/h
    label_norm = (nxmin,nxmax, nymin, nymax)
    
    data.append(norm_load_image_arr)
    output.append(label_norm)


split into test and train data

In [66]:
x = np.array(data, dtype=np.float32)
y = np.array(output,dtype=np.float32)

In [1]:
x.shape, y.shape

NameError: name 'x' is not defined

In [68]:
x_train, x_test, y_train, y_test = train_test_split(x,y,train_size=0.8,random_state=0)

DeepLearning Model

In [74]:
from keras.applications import  MobileNetV2, InceptionV3, InceptionResNetV2
import keras as kr
from keras.layers import Dense, Dropout, Flatten, Input
from keras.models import Model
import tensorflow as tf

In [73]:
inception_resnet = InceptionResNetV2(weights="imagenet", include_top=False, input_tensor=Input(shape=(224,224,3)))
inception_resnet.trainable=False

headmodel = inception_resnet.output
headmodel = Flatten()(headmodel)
headmodel = Dense(500,activation="relu")(headmodel)
headmodel = Dense(250,activation="relu")(headmodel)
headmodel = Dense(4,activation='sigmoid')(headmodel)

model = Model(inputs=inception_resnet.input, outputs=headmodel)

Compiling model

In [75]:
model.compile(loss='mse',optimizer=kr.optimizers.Adam(learning_rate=1e-4))
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv2d_203 (Conv2D)         (None, 111, 111, 32)         864       ['input_2[0][0]']             
                                                                                                  
 batch_normalization_203 (B  (None, 111, 111, 32)         96        ['conv2d_203[0][0]']          
 atchNormalization)                                                                               
                                                                                                  
 activation_203 (Activation  (None, 111, 111, 32)         0         ['batch_normalization_203[

Model training

In [76]:
from keras.callbacks import TensorBoard

In [77]:
tfb = TensorBoard('object_deduction')

In [79]:
history = model.fit(x=x_train, y=y_train, batch_size=10, epochs=100,validation_data=(x_test,y_test),callbacks=[tfb])

Epoch 1/100
35/35 [==============================] - 49s 1s/step - loss: 0.1151 - val_loss: 0.0845
Epoch 2/100
35/35 [==============================] - 33s 953ms/step - loss: 0.0903 - val_loss: 0.1035
Epoch 3/100
35/35 [==============================] - 34s 983ms/step - loss: 0.0948 - val_loss: 0.0984
Epoch 4/100
35/35 [==============================] - 35s 996ms/step - loss: 0.1011 - val_loss: 0.1327
Epoch 5/100
35/35 [==============================] - 36s 1s/step - loss: 0.0967 - val_loss: 0.0933
Epoch 6/100
35/35 [==============================] - 36s 1s/step - loss: 0.0908 - val_loss: 0.1017
Epoch 7/100
35/35 [==============================] - 36s 1s/step - loss: 0.0833 - val_loss: 0.0945
Epoch 8/100
35/35 [==============================] - 36s 1s/step - loss: 0.0917 - val_loss: 0.1094
Epoch 9/100
35/35 [==============================] - 37s 1s/step - loss: 0.0834 - val_loss: 0.0985
Epoch 10/100
35/35 [==============================] - 37s 1s/step - loss: 0.0867 - val_loss: 0.0859


In [80]:
history = model.fit(x=x_train, y=y_train, batch_size=10, epochs=200,validation_data=(x_test,y_test),callbacks=[tfb], initial_epoch=100)


Epoch 101/200
35/35 [==============================] - 42s 1s/step - loss: 0.0341 - val_loss: 0.0400
Epoch 102/200
35/35 [==============================] - 37s 1s/step - loss: 0.0344 - val_loss: 0.0397
Epoch 103/200
35/35 [==============================] - 37s 1s/step - loss: 0.0342 - val_loss: 0.0403
Epoch 104/200
35/35 [==============================] - 38s 1s/step - loss: 0.0345 - val_loss: 0.0400
Epoch 105/200
35/35 [==============================] - 38s 1s/step - loss: 0.0341 - val_loss: 0.0395
Epoch 106/200
35/35 [==============================] - 40s 1s/step - loss: 0.0339 - val_loss: 0.0418
Epoch 107/200
35/35 [==============================] - 46s 1s/step - loss: 0.0344 - val_loss: 0.0397
Epoch 108/200
35/35 [==============================] - 39s 1s/step - loss: 0.0340 - val_loss: 0.0397
Epoch 109/200
35/35 [==============================] - 38s 1s/step - loss: 0.0339 - val_loss: 0.0402
Epoch 110/200
35/35 [==============================] - 39s 1s/step - loss: 0.0348 - val_los

In [82]:
model.save("models\\plate_detection_2.keras")